In [1]:
import nltk
nltk.download('popular')
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import re

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [2]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [3]:
data = pd.read_csv("/content/mit_plagiarism_detection_dataset (1).txt", sep='\t') # Specify the delimiter as tab ('\t')
data.head()

,A person on a horse jumps over a broken down airplane.,"A person is at a diner, ordering an omelette.",0
0,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
1,Children smiling and waving at camera,There are children present,1
2,Children smiling and waving at camera,The kids are frowning,0
3,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0
4,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,1


In [4]:
print(data.columns)
print(data.isnull().sum())  # Find missing values


Index(['A person on a horse jumps over a broken down airplane.',
       'A person is at a diner, ordering an omelette.', '0'],
      dtype='object')
A person on a horse jumps over a broken down airplane.    0
A person is at a diner, ordering an omelette.             4
0                                                         0
dtype: int64


In [5]:
data['0'].value_counts()

,count
0,
0,183965
1,183407


In [6]:
data.shape

(367372, 3)

In [7]:
data = data.rename(columns={'A person on a horse jumps over a broken down airplane.':'person',
                            'A person is at a diner, ordering an omelette.': 'omelette'})
data.head()

,person,omelette,0
0,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
1,Children smiling and waving at camera,There are children present,1
2,Children smiling and waving at camera,The kids are frowning,0
3,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0
4,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,1


In [8]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess_text(text):
    # Check if text is a string to handle potential NaN values
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\W', ' ', text)  # Remove special characters
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text
    else:
        # Handle NaN or other non-string values, e.g., return an empty string
        return ''

# Apply preprocessing to the columns
data['person'] = data['person'].apply(preprocess_text)
data['omelette'] = data['omelette'].apply(preprocess_text)

In [10]:
data

,person,omelette,0
0,person horse jumps broken airplane,person outdoors horse,1
1,children smiling waving camera,children present,1
2,children smiling waving camera,kids frowning,0
3,boy jumping skateboard middle red bridge,boy skates sidewalk,0
4,boy jumping skateboard middle red bridge,boy skateboarding trick,1
...,...,...,...
367367,dog blue collar plays ball outside,dog outside,1
367368,four dirty barefooted children,four children dirty feet,1
367369,four dirty barefooted children,four kids awards cleanest feet,0
367370,man surfing bodysuit beautiful blue water,man business suit heading board meeting,0


In [11]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data["person"] + " " + data["omelette"])

In [12]:
y = data['0']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix")
print(cm)

Accuracy: 0.6901667233752977
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.65      0.68     36957
           1       0.67      0.73      0.70     36518

    accuracy                           0.69     73475
   macro avg       0.69      0.69      0.69     73475
weighted avg       0.69      0.69      0.69     73475

Confusion Matrix
[[24099 12858]
 [ 9907 26611]]


Random Forest

In [15]:
from sklearn.naive_bayes import MultinomialNB
# Instantiate the model
model = MultinomialNB()
# Fit the model
model.fit(X_train, y_train)
# Make predictions
y_pred = model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
# Generate classification report
classification_rep = classification_report(y_test, y_pred)
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Print results
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.644899625723035
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.58      0.62     36957
           1       0.63      0.71      0.67     36518

    accuracy                           0.64     73475
   macro avg       0.65      0.65      0.64     73475
weighted avg       0.65      0.64      0.64     73475

Confusion Matrix:
[[21456 15501]
 [10590 25928]]


In [16]:
import pickle

pickle.dump(model,open("model.pkl",'wb'))
pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl','wb'))

In [17]:
model = pickle.load(open('model.pkl','rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [18]:
def detect(input_text):
    vectorized_text = tfidf_vectorizer.transform([input_text])
    result = model.predict(vectorized_text)
    return "Plagiarim Detected" if result[0] == 1 else "No Plagiarism"

In [19]:
# example ( it is a plagarized text)
input_text = 'Researchers have discovered a new species of butterfly in the Amazon rainforest.'
detect(input_text)

'No Plagiarism'

In [20]:
# example ( it has no plagiarism)
input_text = 'Playing musical instruments enhances creativity.'
detect(input_text)


'Plagiarim Detected'

In [21]:
# example ( it has no plagarism)
input_text = 'Practicing yoga enhances physical flexibility.'
detect(input_text)

'Plagiarim Detected'

In [22]:
# sklearn version
import sklearn
sklearn.__version__

'1.6.1'